<h1 style="color: purple">Script to transfer Excel data into MySQL database</h1>

<h3 style="color: blue">Importing necessary libraries in our program</h3>
<p><em> In our program, we use two libraries, one is Pandas and other is mysql-connector-python
 </em>  </p>

In [188]:
import pandas as pd
import mysql.connector

<h3 style="color: blue">Reading the beginner_assignment01.xlsx file</h3>
<p><em>As we can observe that our excel workbook have two separate sheets so we create two different dataframes for the purpose
</em></p>

In [189]:
df1 = pd.read_excel("beginner_assignment01.xlsx", 'group_listing')
df2 = pd.read_excel("beginner_assignment01.xlsx", 'product_listing')

<h3 style="color: blue">Showing first 5 rows of our dataframe</h3>
<p><em>We check whether we have obtained and are reading the correct data-set</em></p>

In [190]:
df1.head()

,group name,group description,isActive
0,Group 1,Group Number 1,yes
1,Group 2,Group Number 2,yes
2,Group 3,Group Number 3,yes
3,Group 4,Group Number 4,yes
4,Group 5,Group Number 5,yes


In [191]:
df2.head()

,Product Name,Model Name,Product Serial No,Group Associated,product MRP (rs.)
0,Demo Product 1,Product Model 1,98765987659801,Group 1,1001
1,Demo Product 1,Product Model 1,98765987659802,Group 2,1002
2,Demo Product 1,Product Model 1,98765987659803,Group 3,1003
3,Demo Product 1,Product Model 1,98765987659804,Group 4,1004
4,Demo Product 1,Product Model 1,98765987659805,Group 5,1005


<h3 style="color: blue">Converting our data frame to list data structure</h3>
<p><em> This has been done so that it becomes easier to transfer the data given in the excel sheet to our MySQL database which we will see in the upcoming codes
    </em>  </p>

In [192]:
product_list = df2.values.tolist()
group_list = df1.values.tolist()

<h3 style="color: blue">Connecting to our database</h3>
<p><em>We can connect to our database in the below given manner by mentioning host name, user name, password. auth_plugin is optional. In the below script, just add the your respective MySQL host, user and your password
    </em>  </p>

In [193]:
db = mysql.connector.connect(host='localhost', user='root', passwd='password', 
                              auth_plugin='mysql_native_password')

<h3 style="color: blue">Creating cursor</h3>
<p><em>A cursor is a temporary area where all the sql statement execution takes place.
    </em>  </p>

In [194]:
mycursor = db.cursor()

<h3 style="color: blue">Dropping schema</h3>
<p><em>If someone uses this program code and they already have a DB schema named `mydukan` then, it will be dropped for creation of new schema
    </em>  </p>

In [195]:
mycursor.execute("DROP SCHEMA IF EXISTS `mydukan`")

<h3 style="color: blue">Creating DB Schema</h3>
<p><em>We use create statement in order to create the DB Schema named 'mydukan'
    </em>  </p>

In [196]:
mycursor.execute("CREATE SCHEMA `mydukan`")
mycursor.execute("USE `mydukan`")

<h3 style="color: blue">Dropping table</h3>
<p><em>Now we going to create two tables but if somehow, the below mentioned table already exists then, this statement will drop them
    </em>  </p>

In [197]:
mycursor.execute("DROP TABLE IF EXISTS `mydukan.group_listing`")
mycursor.execute("DROP TABLE IF EXISTS `mydukan.product_listing`")

<h3 style="color: blue">Creating new table 'group_listing'</h3>
<p><em>We create a new table 'group_listing' with the below mentioned attributes similar to that of the excel sheet. The column 'group_name' acts as the primary key for the table
    </em>  </p>
 <ul>
    <li>group_name <b>(PRIMARY KEY)</b></li>
    <li>group_description</li>
    <li>isActive</li>
    </ul>

In [198]:
sql = "create table mydukan.group_listing(group_name varchar(20) NOT NULL, group_description varchar(20), isActive varchar(10), \
PRIMARY KEY(group_name))"

In [199]:
mycursor.execute(sql)

<h3 style="color: blue">Inserting into table 'group_listing'</h3>
<p><em>Once, table has been created, we insert the data from list 'group_list' into the table 'group_listing' using the insert statement
    </em>  </p>

In [200]:
sql = "insert into mydukan.group_listing (group_name, group_description, isActive) values (%s, %s, %s)"

In [201]:
for i in group_list:
    g_nm, g_desc, isact = i
    val = (g_nm, g_desc, isact)
    mycursor.execute(sql, val)

<h3 style="color: blue">Creating another table 'product_listing'</h3>
<p><em>We create another table 'product_listing' with the below mentioned attributes similar to that of the excel sheet. The column 'product_serial_no' is the primary key while the column 'group_assc' acts as foreing key referencing 'group_name' column of table 'group_listing'
    </em>  </p>
    
 <ul>
    <li>product_serial_no<b> (PRIMARY KEY)</b></li>
    <li>product_name</li>
    <li>model_name</li>
    <li>group_assc<b> (FOREIGN KEY)</b></li>
    <li>MRP</li>
    </ul>

In [202]:
sql = "create table mydukan.product_listing(product_serial_no varchar(20) NOT NULL, product_name varchar(20), model_name varchar(20), \
group_assc varchar(10), MRP int NOT NULL, PRIMARY KEY(product_serial_no), FOREIGN KEY(group_assc) REFERENCES \
mydukan.group_listing(group_name))"

In [203]:
mycursor.execute(sql)

<h3 style="color: blue">Inserting data into table 'product_listing'</h3>
<p><em>Now, we insert the data from the list 'product_list' into table 'product_listing'
    </em>  </p>

In [204]:
sql = "insert into mydukan.product_listing (product_serial_no, product_name, model_name, \
group_assc, MRP) values (%s, %s, %s, %s, %s)"

In [205]:
for i in product_list:
    p_nm, m_nm, s_no, grp_assc, mrp = i
    val = (s_no, p_nm, m_nm, grp_assc, mrp)
    mycursor.execute(sql, val)

<h3 style="color: blue">Commiting our database</h3>
<p><em>Once both the tables have been created and the data has been inserted, the last and the most important thing we have to do is commit the transaction that took place in order to see the effects.
    </em>  </p>

In [206]:
db.commit()